# Example: Apple 2020

First, lets load the logger:

In [1]:
import logging
from openesef.util.util_mylogger import setup_logger 
logger = setup_logger("main", logging.INFO, log_dir="/tmp/log/")

Now, lets load the xbrl filing, using Apple 2020 as an example:

In [2]:
from openesef.edgar.loader import load_xbrl_filing
#xid, tax = load_xbrl_filing(ticker="AAPL", year=2020)
xid, tax = load_xbrl_filing(filing_url="/Archives/edgar/data/320193/0000320193-20-000096.txt")

The xbrl filing is loaded and the taxonomy is created.

Now, lets print the XBRL instance (xid):

In [3]:
print(xid)


Namespaces: 10
Schema references: 1
Linkbase references: 0
Contexts: 318
Units: 9
Facts: 1388
Footnotes: 0
Filing Indicators: 0


And the taxonomy (tax):

In [4]:
print(tax)

Schemas: 14
Linkbases: 7
Role Types: 775
Arcrole Types: 0
Concepts: 18293
Item Types: 52
Tuple Types: 0
Simple Types: 0
Labels: 1231
References: 0
Hierarchies: 231
Dimensional Relationship Sets: 231
Dimensions: 295
Hypercubes: 363
Enumerations: 0
Enumerations Sets: 0
Table Groups: 0
Tables: 0
Parameters: 0
Assertion Sets: 0
Value Assertions: 0
Existence Assertions: 0
Consistency Assertions: 0


DEI stands for Document and Entity Information. For each XBRL report, there will be a section for DEI,
and this class is to provide easy access to those commonly-defined DEI attributes.

Lets print the DEI:

In [5]:
for i, (key, value) in enumerate(xid.dei.items()):
    print(f"{i}: {key}: {value}")


0: AmendmentFlag: false
1: DocumentFiscalYearFocus: 2020
2: DocumentFiscalPeriodFocus: FY
3: EntityCentralIndexKey: 0000320193
4: CurrentFiscalYearEndDate: --09-26
5: DocumentType: 10-K
6: DocumentAnnualReport: true
7: DocumentPeriodEndDate: 2020-09-26
8: DocumentTransitionReport: false
9: EntityFileNumber: 001-36743
10: EntityRegistrantName: Apple Inc.
11: EntityIncorporationStateCountryCode: CA
12: EntityTaxIdentificationNumber: 94-2404110
13: EntityAddressAddressLine1: One Apple Park Way
14: EntityAddressCityOrTown: Cupertino
15: EntityAddressStateOrProvince: CA
16: EntityAddressPostalZipCode: 95014
17: CityAreaCode: 408
18: LocalPhoneNumber: 996-1010
19: TradingSymbol: AAPL
20: EntityWellKnownSeasonedIssuer: Yes
21: EntityVoluntaryFilers: No
22: EntityCurrentReportingStatus: Yes
23: EntityInteractiveDataCurrent: Yes
24: EntityFilerCategory: Large Accelerated Filer
25: EntitySmallBusiness: false
26: EntityEmergingGrowthCompany: false
27: IcfrAuditorAttestationFlag: true
28: EntitySh

Lets explore the presentation networks defined in the linkbase:

In [6]:
from openesef.util.parse_concepts import get_presentation_networks
get_presentation_networks(tax)


Accessing presentation networks...
Number of base sets: 231

Found presentation base set: presentationArc|http://www.xbrl.org/2003/arcrole/parent-child|http://www.apple.com/role/CoverPage

Found presentation base set: presentationArc|http://www.xbrl.org/2003/arcrole/parent-child|http://www.apple.com/role/CONSOLIDATEDSTATEMENTSOFOPERATIONS

Found presentation base set: presentationArc|http://www.xbrl.org/2003/arcrole/parent-child|http://www.apple.com/role/CONSOLIDATEDSTATEMENTSOFCOMPREHENSIVEINCOME

Found presentation base set: presentationArc|http://www.xbrl.org/2003/arcrole/parent-child|http://www.apple.com/role/CONSOLIDATEDBALANCESHEETS

Found presentation base set: presentationArc|http://www.xbrl.org/2003/arcrole/parent-child|http://www.apple.com/role/CONSOLIDATEDBALANCESHEETSParenthetical

Found presentation base set: presentationArc|http://www.xbrl.org/2003/arcrole/parent-child|http://www.apple.com/role/CONSOLIDATEDSTATEMENTSOFSHAREHOLDERSEQUITY

Found presentation base set: pres

[Arc name: presentationArc
 Arc role: http://www.xbrl.org/2003/arcrole/parent-child
 Role: http://www.apple.com/role/CoverPage
 Roots: dei:CoverAbstract,
 Arc name: presentationArc
 Arc role: http://www.xbrl.org/2003/arcrole/parent-child
 Role: http://www.apple.com/role/CONSOLIDATEDSTATEMENTSOFOPERATIONS
 Roots: us-gaap:IncomeStatementAbstract,
 Arc name: presentationArc
 Arc role: http://www.xbrl.org/2003/arcrole/parent-child
 Role: http://www.apple.com/role/CONSOLIDATEDSTATEMENTSOFCOMPREHENSIVEINCOME
 Roots: us-gaap:StatementOfIncomeAndComprehensiveIncomeAbstract,
 Arc name: presentationArc
 Arc role: http://www.xbrl.org/2003/arcrole/parent-child
 Role: http://www.apple.com/role/CONSOLIDATEDBALANCESHEETS
 Roots: us-gaap:StatementOfFinancialPositionAbstract,
 Arc name: presentationArc
 Arc role: http://www.xbrl.org/2003/arcrole/parent-child
 Role: http://www.apple.com/role/CONSOLIDATEDBALANCESHEETSParenthetical
 Roots: us-gaap:StatementOfFinancialPositionAbstract,
 Arc name: presentat